In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [2]:
path='/home/drallab/datamining/Yelp_Sentiment_Analysis-master/data/review_subset.csv'
data = pd.read_csv(path)
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4781551 entries, 0 to 4781550
Data columns (total 6 columns):
business_id    object
date           object
review_id      object
stars          float64
text           object
user_id        object
dtypes: float64(1), object(5)
memory usage: 218.9+ MB


In [3]:
df1 = data.iloc[:,3:5]

In [4]:
df1.info()
df1 = df1[(df1.stars == 5) | (df1.stars == 1)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4781551 entries, 0 to 4781550
Data columns (total 2 columns):
stars    float64
text     object
dtypes: float64(1), object(1)
memory usage: 73.0+ MB


In [5]:
df1 = shuffle(df1)

In [6]:
df1.dropna(inplace=True)

In [7]:
df1.head()

,stars,text
2038099,5.0,Wow this place is. gem. Best oysters i ever ha...
3103183,5.0,Wish there were more stars to give this amazin...
4024800,5.0,"Everything is amazing here, the line is out th..."
3031605,5.0,just got text msg they have re-opened!!! 1/1...
3442138,5.0,"Hello friends,\n\nCall in your order, gets kin..."


In [8]:

X_data = df1['text']
y_data = df1['stars']

In [9]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test= train_test_split(X_data,y_data,test_size =0.25,random_state=42)
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(2269756,)
(756586,)
(2269756,)
(756586,)


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm

<2269756x354985 sparse matrix of type '<type 'numpy.int64'>'
	with 146694160 stored elements in Compressed Sparse Row format>

In [11]:
X_test_dtm = vect.transform(X_test)

In [12]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y_train=lb.fit_transform(y_train)
y_test=lb.fit_transform(y_test)

In [13]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=42)

In [14]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [15]:
rfc1=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 100, max_depth=5, criterion='gini')

In [16]:
rfc1.fit(X_train_dtm, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [17]:
pred=rfc1.predict(X_test_dtm)

In [18]:
from sklearn.metrics import accuracy_score
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))

('Accuracy for Random Forest on CV data: ', 0.753905041859088)


In [20]:
#Lets see the confusion metrix 

from sklearn import metrics

conf_matrix=metrics.confusion_matrix(y_test,pred)
print('confusion matrix \n',conf_matrix)

('confusion matrix \n', array([[     8, 186192],
       [     0, 570386]]))


In [22]:
pd.crosstab(y_test, pred, rownames=['True'], colnames=['Predicted'], margins=True)


Predicted,0,1,All
True,,,
0,8,186192,186200
1,0,570386,570386
All,8,756578,756586


In [25]:
auc_val = metrics.roc_auc_score(y_test,pred)
print('ROC_AUC_SCORE:- ', auc_val)


('ROC_AUC_SCORE:- ', 0.5000214822771214)


In [27]:
#Sensitivity : is the classifier to detecting positive instances , also True Positive Rate" or "Recall"
recall_NB= metrics.recall_score(y_test,pred)
print ('Recall score',recall_NB)


('Recall score', 1.0)


In [28]:
#Precision: When a positive value is predicted, how often is the prediction correct?
#How "precise" is the classifier when predicting positive instances?

Precision_NB= metrics.precision_score(y_test,pred)
print('Precision Score', Precision_NB)

('Precision Score', 0.7539024396691418)


In [29]:
#print rating 1 reviews which are falsely callisified as positive i.e rating 5 ,diplay False Postive review comments
X_test[(pred == 1) & (y_test == 0)]

4432762    They got my order wrong and I called multiple ...
3475768    Rude.  Stupid.  Horrible service and yells bac...
1694768    This place wasn't worth the drive... Was promi...
2791753    We wanted to grab a quick dinner with several ...
1980912    The entertainment was enjoyable. The food, was...
2753000    Awful service. I was really counting on them a...
4131638    My first couple of experiences were good. They...
3255715    literally no stars. i called in a week ago, ma...
4067926    Just a bad place \n"Good morning " none of the...
2853602    New ownership company from New York. Lack of p...
33828      This was our 2nd time in this week....the 1st ...
4370941    Awful experience.  You can't teach people to c...
2631506    Average food and atomsphere with a very rude a...
4165860    Got here around 1pm for brunch on a Sunday, wa...
2250981    Don't order from here if you actually want you...
4622658    What a unfriendly place! We walked in and was ...
3590101    Went today an

In [30]:
#print rating 5 reviews which are falsely callisified as negative i.e rating 1,diplay True Negative review comments
X_test[(pred == 0) & (y_test == 1)]

Series([], Name: text, dtype: object)

In [31]:
#Lets print classifcation report
class_report = metrics.classification_report(y_test,pred)
print (class_report)

              precision    recall  f1-score   support

           0       1.00      0.00      0.00    186200
           1       0.75      1.00      0.86    570386

   micro avg       0.75      0.75      0.75    756586
   macro avg       0.88      0.50      0.43    756586
weighted avg       0.81      0.75      0.65    756586

